In [32]:
import json
import datetime
import pandas as pd

# 팍스넷에서 데이터 주어옴. http://www.paxnet.co.kr/stock/analysis/chart?abbrSymbol=114800
# html js코드에 json 문자열이 그대로 붙어있음. 긁어와서 json으로 저장하면 끝~

# file_name = "data_kodex_kospi"
# rename_prefix = "KDX"

file_name = "data_kodex_kospi_inverse"
rename_prefix = "KDI"

str_data = open("{}.json".format(file_name), "r").readlines()[0]
str_data = str_data.replace("\n", "").replace("\\","")
data = json.loads(str_data)
df_src = pd.DataFrame(data["body"]["chartDataList"])

In [33]:
df = df_src.copy()
date_list = [str(datetime.datetime.fromtimestamp(x)).split(" ")[0] for x in df["tradeDt"]]
time_list = [str(datetime.datetime.fromtimestamp(x)).split(" ")[1].split(":")[0] for x in df["tradeDt"]]
df["date"] = date_list
df["time"] = time_list
df["datetime"] = df["date"].str.replace("-", "") + df["time"]
df = df[['date', 'time', 'datetime', 'openPrice', 'closePrice', 'highPrice', 'lowPrice', 'volume']]
df.rename(columns={
    "date" : "DATE",
    "time" : "TIME",
    "datetime" : "DATETIME",    
    "openPrice" : "{}_OPN".format(rename_prefix),
    "closePrice" : "{}_CLS".format(rename_prefix),
    "highPrice" : "{}_HIGH".format(rename_prefix),
    "lowPrice" : "{}_LOW".format(rename_prefix),
    "volume" : "{}_VOL".format(rename_prefix)
}, inplace=True)
df.to_excel("{}.xlsx".format(file_name), index=False)
df

,DATE,TIME,DATETIME,KDI_OPN,KDI_CLS,KDI_HIGH,KDI_LOW,KDI_VOL
0,2021-09-24,11,2021092411,3915,3925,3925,3900,5804180.0
1,2021-09-24,12,2021092412,3925,3920,3930,3915,4590549.0
2,2021-09-24,13,2021092413,3920,3925,3930,3915,2754191.0
3,2021-09-24,14,2021092414,3925,3920,3930,3920,1777032.0
4,2021-09-24,15,2021092415,3920,3920,3930,3920,2311850.0
...,...,...,...,...,...,...,...,...
995,2022-04-22,12,2022042212,4405,4410,4410,4395,2076584.0
996,2022-04-22,13,2022042213,4405,4400,4410,4400,969359.0
997,2022-04-22,14,2022042214,4405,4395,4410,4395,1958333.0
998,2022-04-22,15,2022042215,4395,4395,4400,4385,2278272.0
